In [1]:
import numpy as np
from Temporal_Community_Detection import temporal_network
from helpers import *
import matplotlib.pyplot as plt
import pickle
from math import floor
import random
import csv
import matplotlib as mpl

/Users/bengieru/MLN/Results/Calcium_test/Temporal_Community_Detection.py:29: UserWarning: Graph-tool requires its own environment. Restart the kernel with a gt environment to run DSBM, otherwise you can proceed.
  warnings.warn(message = 'Graph-tool requires its own environment. Restart the kernel with a gt environment to run DSBM, otherwise you can proceed.')


In [2]:
np.linspace(0.9,1.3,9)

array([0.9 , 0.95, 1.  , 1.05, 1.1 , 1.15, 1.2 , 1.25, 1.3 ])

In [ ]:
def remove_non_firing(spikes, thresh, pv):
    n,t = spikes.shape
    indices = []
    for i in range(n):
        if len(np.nonzero(spikes[i])[0]) >= thresh:
            indices.append(i)
    removed_spikes = np.zeros((len(indices),t))
    for i,e in enumerate(indices):
        removed_spikes[i] = spikes[e]
    removed_neurons = list(set([i for i in range(n)])-set(indices))
    
    ##update pv indices
    new_pvs = np.array(sorted(pv))
    for i,e in enumerate(sorted(removed_neurons)):
        new_pvs = np.append(new_pvs[new_pvs<e],new_pvs[new_pvs>e]-1, axis = 0)
    list(new_pvs)
    return(removed_spikes, removed_neurons, new_pvs)

In [ ]:
path = '/projects/academic/smuldoon/bengieru/Community_Detection/calcium_data_test/' ## base path
output = 'Johan_Clean_Traces_Features_and_Spikes/' #spikes and traces file
roi = 'sarah_ROI/' #roi file
subjects = load_obj(path + 'subjects/', 'subjects')
epochs = ['_baseline', '_early', '_pre']
trackable = [subjects['wt'][3], subjects['het'][6], subjects['het'][8], subjects['het'][13], subjects['het'][4], subjects['het'][3], subjects['het'][2]]

pvs = {}
pvs['%s'%trackable[0]] = load_obj(path +'subjects/','pv_wt')['%s'%trackable[0]]
for i in range(1,len(trackable)):
    temp = load_obj(path +'subjects/','pv_het')['%s'%trackable[i]]
    pvs['%s'%trackable[i]] = (np.array(temp)-1).tolist() ##adjust python indexing

In [ ]:
time = 2000 ## binning the time into chunks of
layers = int(8000/time) ## number of layers
spikes_trackable = {}
num_rois_trackable = {}
for i,e in enumerate(trackable):
    for j,f in enumerate(epochs):
        spikes_trackable['%s'%(e+f)] = read_csv(path, output, e+f, roi)
        num_rois_trackable['%s'%(e+f)] = read_roi(path, roi, e+f) ## number of rois

In [ ]:
new_spikes_trackable ={}
#remove non-firing
for i,e in enumerate(trackable):
    for j,f in enumerate(epochs):
        spikes_trackable, removed_neurons, new_pvs = remove_non_firing(spikes_trackable['%s'%(e+f)], 5 ,pvs['%s'%e][0])
        path_nbr_update = path + 'infomap/neighborhood_update/%s/'%(e)
        os.makedirs(path_nbr_update, exist_ok = True)
        with open(path_nbr_update + "firing_spikes_%s.pkl"%(e+f), "wb") as fp:
                pickle.dump(new_spikes_trackable, fp)
        with open(path_nbr_update + "adjusted_pvs_%s.pkl"%(e+f), "wb") as fp:
                pickle.dump(new_pvs, fp)
        with open(path_nbr_update + "removed_neurons_%s.pkl"%(e+f), "wb") as fp:
                pickle.dump(removed_neurons, fp)
        new_spikes_trackable['%s'%(e+f)] = spikes_trackable
        num_rois_trackable['%s'%(e+f)] = int(num_rois_trackable['%s'%(e+f)] - len(removed_neurons))

In [ ]:
binned_binary_spikes_trackable = {}
for i,e in enumerate(trackable):
    path_nbr_update = path + 'infomap/neighborhood_update/%s/'%(e)
    for j,f in enumerate(epochs):
        # bin the spikes into fixed length and apply gaussian kernel of length 9
        binary_trackable = binarize(new_spikes_trackable['%s'%(e+f)])
        binned_binary_spikes_trackable['%s'%(e+f)] = bin_time_series(binary_trackable, 
                                                                     time, 
                                                                     gaussian = True, 
                                                                     sigma = 1.25)
        
        with open(path_nbr_update + "binary_spikes_%s.pkl"%(e+f), "wb") as fp:
            pickle.dump(binary_trackable, fp)

In [ ]:
adjacency_matrices = {}
for i,e in enumerate(trackable):
    path_nbr_update = path + 'infomap/neighborhood_update/%s/'%(e)
    for j,f in enumerate(epochs):  
        adjacencies = []
        for k in range(layers):
            adjacencies.append(cross_correlation_matrix(binned_binary_spikes_trackable['%s'%(e+f)][k])[0])
        adjacency_matrices['%s'%(e+f)] = adjacencies
        
        fig,ax = plt.subplots(1,4, figsize = (60,20))
        for i in range(4):
            k = ax[i].imshow(adjacency_matrices['%s'%(e+f)][i], 
                             origin = 'lower', 
                             interpolation = 'nearest', 
                             aspect='auto',  
                             extent = [0,num_rois_trackable['%s'%(e+f)],0,num_rois_trackable['%s'%(e+f)]])
            ax[i].set_title('Adjacency Matrix (Layer %d)'%(i +1), fontsize = 30)
            ax[i].set_xticks([k*10 for k in range(int(num_rois_trackable['%s'%(e+f)]/10)+1)])
            ax[i].set_yticks([k*10 for k in range(int(num_rois_trackable['%s'%(e+f)]/10)+1)])
            ax[i].tick_params(axis = 'both', labelsize = 25)
        cbar = fig.colorbar(k, ax = ax.flat, orientation = 'horizontal')
        cbar.ax.tick_params(labelsize = 30) 
        plt.savefig(path_nbr_update + 'adjacencies_%s.pdf'%(e+f))

In [ ]:
TNs = {}
for i,e in enumerate(trackable):
    for j,f in enumerate(epochs): 
        TNs['%s'%(e+f)] = temporal_network(num_rois_trackable['%s'%(e+f)], 
                                    layers, 
                                    time, 
                                    data = 'list__adjacency', 
                                    list_adjacency = adjacency_matrices['%s'%(e+f)], 
                                    omega = 1, 
                                    kind = 'ordinal')

In [ ]:
threshs = np.linspace(0.1, 0.5, 9)
inters = np.linspace(0, 0.8, 9)
for i,e in enumerate(trackable):
    path_nbr_update = path + 'infomap/neighborhood_update/%s/'%(e)
    for j,f in enumerate(epochs):
        membership_nbr_update, labels_nbr_update = TNs['%s'%(e+f)].run_community_detection('infomap', 
                                                                                           update_method = 'neighborhood', 
                                                                                           interlayers = inters, 
                                                                                           thresholds = threshs)
        with open(path_nbr_update + "labels_%s.pkl"%(e+f), "wb") as fp:
            pickle.dump(labels_nbr_update, fp)
        
        with open(path_nbr_update + "membership_%s.pkl"%(e+f), "wb") as fp:
            pickle.dump(membership_nbr_update, fp)
            
        fig,ax = plt.subplots(9,9, figsize = (9*15+5,9*15))
        for i in range(9):
            for j in range(9):
                comms, c = TNs['%s'%(e+f)].community(membership_nbr_update['interlayer=%.3f'%inters[i]][j], ax[i][j])
                ax[i][j].set_xticks([i for i in range(layers)])
                ax[i][j].set_yticks([i*10 for i in range(int(num_rois_trackable['%s'%(e+f)]/10)+1)])
                ax[i][j].tick_params(axis = 'both', labelsize = 12)
                ax[i][j].set_xlabel('Layers (Time)', fontsize = 25)
                ax[i][j].set_ylabel('Neuron ID', fontsize = 25)
                ax[i][j].set_title('%d Communities, interlayer:%.3f, threshold:%.3f'%(len(c),inters[i],threshs[j]), fontsize=29)
        plt.tight_layout()
        plt.savefig(path_nbr_update + 'communities_%s.pdf'%(e+f))